# 1. 选择文件夹

In [1]:
import os

In [2]:
list_file = list(os.walk('./'))[0][2]
for file in list_file:
    if '替代' in file:
        alternate = './' + file
    elif '断货改价' in file:
        base = './' + file
    elif '销量表' in file:
        sales = './' + file

print(f'\talternate: \t{alternate}')
print(f'\tbase: \t\t{base}')
print(f'\tsales: \t\t{sales}')

	alternate: 	./替代ok表_20230710_094306.xlsx
	base: 		./color_Lennon_断货改价-us-Ada-上午.xlsx
	sales: 		./销量表4.xlsx


# 2. 替代料表

In [3]:
import pandas as pd

In [4]:
df_alternate = pd.read_excel(alternate,
                             header=0).fillna('')

df_alternate

,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,替换料6
0,USAC64980T,USAC64980,,,,,
1,USACC00055-C,USACC00055,,,,,
2,USACC01207-C,USACC01207,,,,,
3,USACC01311-C,USACC01311,,,,,
4,USACC01330S-C,USACC01330,,,,,
...,...,...,...,...,...,...,...
6176,USUSH300G,USUSH300,,,,,
6177,USUSH350G,USUSH350,,,,,
6178,USVCS11059J,USVCS11059,,,,,
6179,USVCS11064U,USVCS11064,,,,,


In [5]:
df_alternate['row'] = [str(i) for i in range(len(df_alternate))]

df_alternate

,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,替换料6,row
0,USAC64980T,USAC64980,,,,,,0
1,USACC00055-C,USACC00055,,,,,,1
2,USACC01207-C,USACC01207,,,,,,2
3,USACC01311-C,USACC01311,,,,,,3
4,USACC01330S-C,USACC01330,,,,,,4
...,...,...,...,...,...,...,...,...
6176,USUSH300G,USUSH300,,,,,,6176
6177,USUSH350G,USUSH350,,,,,,6177
6178,USVCS11059J,USVCS11059,,,,,,6178
6179,USVCS11064U,USVCS11064,,,,,,6179


In [6]:
list_df = []
for i in ['主料SKU', '替换料1', '替换料2', '替换料3', '替换料4', '替换料5', '替换料6']:
    df_temp =  df_alternate[[i,
                             'row']]
    df_temp.columns = ['SKU',
                       'row']
    df_temp = df_temp[df_temp['SKU'] != '']
    list_df.append(df_temp)

df_position = pd.concat(list_df).reset_index(drop=True)

df_position

,SKU,row
0,USAC64980T,0
1,USACC00055-C,1
2,USACC01207-C,2
3,USACC01311-C,3
4,USACC01330S-C,4
...,...,...
14072,USGS27875,4794
14073,USGS6310,5267
14074,USGS64520,5334
14075,USGS65180,5364


# 3. 销量计算

In [7]:
df = pd.read_excel(base,
                   header=1)[['SKU']]

df

,SKU
0,USFP2332S0Z-C
1,USFP2344M0Z-C
2,USFP2359M0Z-C
3,USFP2442M0Z-C
4,USFP2455M0Z-C
...,...
11145,USGS61110
11146,USGS95983
11147,USITC18068
11148,USITC18079


In [8]:
df = pd.merge(left=df, right=df_position, how='left', left_on='SKU', right_on='SKU').fillna('')
df = df[['SKU',
         'row']]

df

,SKU,row
0,USFP2332S0Z-C,3668
1,USFP2344M0Z-C,3675
2,USFP2359M0Z-C,3688
3,USFP2442M0Z-C,3710
4,USFP2455M0Z-C,3719
...,...,...
11145,USGS61110,
11146,USGS95983,
11147,USITC18068,
11148,USITC18079,


In [9]:
df_sales = pd.read_excel(sales,
                         header=0,
                         usecols='D, F')

df_sales = df_sales.groupby(df_sales['SKU'], as_index=False).sum()

df_sales

,SKU,购买数量
0,USABS00021,1
1,USABS03720,1
2,USABS10870,1
3,USABS10990,1
4,USABS1160,1
...,...,...
1645,USWS10T01,1
1646,USWS59858-C,1
1647,USWS5K495,1
1648,USWS6A663,1


In [10]:
df = pd.merge(left=df, right=df_sales, how='left', left_on='SKU', right_on='SKU').fillna(0)

df

,SKU,row,购买数量
0,USFP2332S0Z-C,3668,0.0
1,USFP2344M0Z-C,3675,0.0
2,USFP2359M0Z-C,3688,0.0
3,USFP2442M0Z-C,3710,0.0
4,USFP2455M0Z-C,3719,0.0
...,...,...,...
11145,USGS61110,,0.0
11146,USGS95983,,0.0
11147,USITC18068,,0.0
11148,USITC18079,,0.0


In [11]:
df_sales = pd.merge(left=df_sales, right=df_position, how='left', left_on='SKU', right_on='SKU').fillna('')
df_sales = df_sales[df_sales['row'] != ''][['row', '购买数量']].copy()
df_sales = df_sales.groupby([df_sales['row']], as_index=False).sum()

df_sales

,row,购买数量
0,1031,1
1,1037,1
2,1096,1
3,1102,2
4,1113,1
...,...,...
340,876,1
341,895,1
342,907,1
343,963,1


In [12]:
df = pd.merge(left=df, right=df_sales, how='left', left_on='row', right_on='row', suffixes=('_sales_1', '_sales_2')).fillna('')

df

,SKU,row,购买数量_sales_1,购买数量_sales_2
0,USFP2332S0Z-C,3668,0.0,
1,USFP2344M0Z-C,3675,0.0,
2,USFP2359M0Z-C,3688,0.0,
3,USFP2442M0Z-C,3710,0.0,
4,USFP2455M0Z-C,3719,0.0,
...,...,...,...,...
11145,USGS61110,,0.0,
11146,USGS95983,,0.0,
11147,USITC18068,,0.0,
11148,USITC18079,,0.0,


In [13]:
df['购买数量'] = [df.loc[i, '购买数量_sales_2'] if df.loc[i, '购买数量_sales_2'] != '' else df.loc[i, '购买数量_sales_1'] for i in range(len(df))]

df = df[['SKU', '购买数量']]

df.to_excel('./date 销量.xlsx', index=False)